In [1]:
import cv2
import numpy as np
import os
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from random import shuffle
import tensornets as nets
from PIL import Image
%matplotlib inline

In [3]:
os.listdir('hackthon-data/ai_train_64/ai_challenger_scene_train/')

['scene_classes.csv', 'scene_train_images', 'scene_labels.csv']

In [5]:
df_a = pd.read_csv('hackthon-data/ai_train_64/ai_challenger_scene_train/scene_labels.csv')

In [6]:
df_a.shape

(53879, 3)

In [7]:
df_a = df_a.loc[df_a.label_id != 'jsessionid=1B3094C4FF247D0E497F91743433395E?r='].reset_index()

In [8]:
len(df_a.label_id.unique())

81

In [9]:
df_a.label_id = df_a.label_id.astype(int)
df_a.drop(['index', 'Unnamed: 0'], axis=1, inplace=True)

In [10]:
df_a.head()

,image_id,label_id
0,79f993ae0858ae238b22968c5934d1ddba585ae4.jpg,66
1,e963208fe9e90df0c385f7367bcdb6d0d5d0b165.jpg,61
2,02df5ecbf7c749ccc9d833f129bbd5d9837940ce.jpg,64
3,5620eb385b7567fb087813cf5233b5ceecdeeca3.jpg,31
4,f8b4d42001a562fc63b9b39c02531661c0e236ca.jpg,19


In [12]:
df_b = pd.read_csv('hackthon-data/ai_train_64/ai_challenger_scene_train/scene_classes.csv', header=None)
df_b.drop(1, axis=1, inplace=True)
df_b.rename({0:'label_id', 2: 'label_name'}, axis=1, inplace=True)

In [13]:
df_b.head()

,label_id,label_name
0,0,airport_terminal
1,1,landing_field
2,2,airplane_cabin
3,3,amusement_park
4,4,skating_rink


In [14]:
df = pd.merge(df_a, df_b, on='label_id', how='inner')

In [15]:
df.head()

,image_id,label_id,label_name
0,79f993ae0858ae238b22968c5934d1ddba585ae4.jpg,66,auto_showroom
1,a74b20339e92d39e00920806f1aa6a3813d9481d.jpg,66,auto_showroom
2,dc389691f5cd2233f1bda34940017b33ff386298.jpg,66,auto_showroom
3,3584d24e091c6f3d79fa1d40f80f2789fac73b46.jpg,66,auto_showroom
4,2a27398da703ac3212858b7f9c442258203ffd8f.jpg,66,auto_showroom


In [16]:
set(df.label_name)

{'airplane_cabin',
 'airport_terminal',
 'amusement_park',
 'aquarium',
 'aqueduct',
 'arena/performance',
 'art_room',
 'assembly_line',
 'athletic_field',
 'auto_showroom',
 'balcony',
 'banquet_hall',
 'bar',
 'baseball_field',
 'basketball_court',
 'bazaar',
 'beauty_salon',
 'bedchamber',
 'bowling_alley',
 'boxing_ring',
 'bridge',
 'campsite',
 'church',
 'classroom',
 'clothing_store',
 'coffee_shop',
 'conference_room',
 'construction_site',
 'countryside',
 'desert/sand',
 'dining_room',
 'discotheque',
 'elevator/staircase',
 'farm/farm_field',
 'firefighting',
 'football_field',
 'forest',
 'garden',
 'gas_station',
 'general_store',
 'golf_course',
 'greenhouse',
 'gymnasium',
 'hospital',
 'igloo/ice_engraving',
 'kitchen',
 'laboratory',
 'lake/river',
 'landfill',
 'landing_field',
 'lawn',
 'library/bookstore',
 'mountain',
 'museum',
 'music_studio',
 'nursery',
 'ocean/beach',
 'office',
 'orchard/vegetable',
 'palace',
 'pasture',
 'pavilion',
 'plaza',
 'racecourse

In [17]:
df.loc[df.label_name == 'airport_terminal', :].head()

,image_id,label_id,label_name
16480,6170cbea8b54c3aac36b3448135a2265a502d6a1.jpg,0,airport_terminal
16481,4da9ecb7c74d6170e6ac6ce20c7cc5a586f9485a.jpg,0,airport_terminal
16482,0281566764968f1527b459bbda402ecf9e20ec40.jpg,0,airport_terminal
16483,544c182febc40b01dce005b760b8dbd291fee718.jpg,0,airport_terminal
16484,f2e160fdfbc3fc716b12629640b02e47f24d8889.jpg,0,airport_terminal


In [ ]:
img1 = mpimg.imread('hackthon-data/ai_challenger_scene_train/scene_train_images/6170cbea8b54c3aac36b3448135a2265a502d6a1.jpg')
imgplot = plt.imshow(img1)
plt.show()

In [ ]:
img = mpimg.imread('hackthon-data/ai_challenger_scene_train/scene_train_images/3584d24e091c6f3d79fa1d40f80f2789fac73b46.jpg')
imgplot = plt.imshow(img)
plt.show()

In [19]:
# img.shape

In [20]:
def indices_to_one_hot(data, nb_classes):
    """
    Convert an iterable of indices to one-hot encoded labels.
    """
    targets = np.array(data).reshape(-1)
    return np.eye(nb_classes)[targets]

In [21]:
ROOT_PATH = "hackthon-data/ai_train_64/ai_challenger_scene_train/scene_train_images"

In [22]:
df.head()

,image_id,label_id,label_name
0,79f993ae0858ae238b22968c5934d1ddba585ae4.jpg,66,auto_showroom
1,a74b20339e92d39e00920806f1aa6a3813d9481d.jpg,66,auto_showroom
2,dc389691f5cd2233f1bda34940017b33ff386298.jpg,66,auto_showroom
3,3584d24e091c6f3d79fa1d40f80f2789fac73b46.jpg,66,auto_showroom
4,2a27398da703ac3212858b7f9c442258203ffd8f.jpg,66,auto_showroom


In [23]:
def next_batch(num, data, labels):
    '''
    Return a total of num random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]
    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [24]:
label = []
images = []
for index, row in df.iterrows():
    label.append(indices_to_one_hot(row['label_id'], 80))
    images.append(cv2.imread(ROOT_PATH + row['image_id']))

In [25]:
images_array = np.array(images)
labels_array = np.array(label)

In [ ]:
# np.save('make_checkpoint_folders/images_array.npy', images_array)
# np.save('make_checkpoint_folders/labels_array.npy', labels_array)

In [26]:
labels_array = labels_array.reshape(53876, 80)

In [27]:
def conv_layer(input_layer, filters, strides, is_training):
    layer = tf.layers.conv2d(
        inputs=input_layer, 
        filters=filters, 
        kernel_size=3,
        strides=strides, 
        padding='same', 
        activation=None,
        kernel_initializer=tf.truncated_normal_initializer()
    )
    
    layer = tf.layers.batch_normalization(
        inputs=layer, 
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True,
    )
    
    layer = tf.nn.relu(layer)
    return layer

def fully_connected(input_layer, num_units, is_training):
    layer = tf.layers.dense(input_layer, num_units, use_bias=False, activation=None)
    layer = tf.layers.batch_normalization(layer)
    layer = tf.nn.relu(layer)
    return layer

def conv_network(feature, label, num_class, image_size, keep_prob):
    input_layer = tf.reshape(feature, [-1, image_size, image_size, 3])
    
    # 16 conv layers with 64, 128, 256, 512 filters.
    layer = conv_layer(input_layer, 64, 1)
    layer = conv_layer(layer, 64, 1, is_training)
    layer = tf.layers.max_pooling2d(layer, pool_size=[2, 2], strides=2, padding='same')
    
    layer = conv_layer(layer, 128, 1)
    layer = conv_layer(layer, 128, 1)
    layer = tf.layers.max_pooling2d(layer, pool_size=[2, 2], strides=2, padding='same')
    
    layer = conv_layer(layer, 256, 2)
    layer = conv_layer(layer, 256, 2)
    layer = conv_layer(layer, 256, 2)
    layer = conv_layer(layer, 256, 2)
    layer = tf.layers.max_pooling2d(layer, pool_size=[2, 2], strides=2, padding='same')
    
    layer = conv_layer(layer, 512, 2)
    layer = conv_layer(layer, 512, 2)
    layer = conv_layer(layer, 512, 2)
    layer = conv_layer(layer, 512, 2)
    layer = tf.layers.max_pooling2d(layer, pool_size=[2, 2], strides=2, padding='same')
    
    layer = conv_layer(layer, 512, 2)
    layer = conv_layer(layer, 512, 2)
    layer = conv_layer(layer, 512, 2)
    layer = conv_layer(layer, 512, 2)
    layer = tf.layers.max_pooling2d(layer, pool_size=[2, 2], strides=2, padding='same')
    
    shape = layer.get_shape().as_list()
    layer = tf.reshape(layer, shape=[-1, shape[1]*shape[2]*shape[3]])
    layer = fully_connected(layer, 800)
    layer = tf.nn.dropout(layer, keep_prob)
    logits = tf.layers.dense(layer, 80)
    output = tf.sigmoid(logits)
    
    # Loss and optimizer
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=label))
    
    return logits, loss

In [28]:
num_class = 80
batch_size = 64
image_size = 64
max_step = 60000
learning_rate =0.002

In [29]:
keep_prob = tf.placeholder(tf.float32)
x = tf.placeholder(tf.float32, shape=(None, image_size, image_size, 3), name='input_x')
y = tf.placeholder(tf.float32, shape=(None, num_class), name='output_y')

In [32]:
logits, loss = conv_network(images_array, labels_array, num_class, image_size, keep_prob)

TypeError: Expected binary or unicode string, got None

In [ ]:
with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
    train_opt = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_label, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
       
    with tf.Session() as sess:
        saver = tf.train.Saver()
        ckpt = tf.train.get_checkpoint_state(checkpoint_path)

In [ ]:
# logits.print_summary()

In [ ]:
# init = tf.global_variables_initializer()
# with tf.Session() as sess:
#     # Run the initializer
#     sess.run(init)
#     for step in range(1, num_steps+1):
#         batch_x, batch_y = next_batch(batch_size,images_array,labels_array)
#         # Run optimization op (backprop)
#         sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: 1.0})
#         if step % display_step == 0 or step == 1:
#             # Calculate batch loss and accuracy
#             loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,Y: batch_y,keep_prob: dropout})
#             print("Step " + str(step) + ", Minibatch Loss= " + \
#                   "{:.4f}".format(loss) + ", Training Accuracy= " + \
#                   "{:.3f}".format(acc))

#     print("Optimization Finished!")

In [ ]:
save_model_path = './make_checkpoint_folders'
saver = tf.train.Saver(max_to_keep=3)
print('Training...')
with tf.Session() as sess:
    ckpt = tf.train.get_checkpoint_state(save_model_path)
    # Initializing the variables
    if ckpt and ckpt.model_checkpoint_path:
        print('Restore the model from checkpoint {}.'.format(ckpt.model_checkpoint_path))
        start_step = int(ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1])
        saver.restore(sess, ckpt.model_checkpoint_path)
        print (start_step)
    else:
        sess.run(tf.global_variables_initializer())
        start_step = 0
        print('Start training from new start.')        
        print('global_variables_initializer ... done ...')
        sess.run(logits.pretrained())
        print('model.pretrained ... done ... ')

    # Training cycle
    print('starting training ... ')
    for step in range(start_step, num_steps+1):
        batch_x, batch_y = next_batch(batch_size,images_array,labels_array)
        # Run optimization op (backprop)
        sess.run(train, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.5})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={x: batch_x,y: batch_y,keep_prob: dropout})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
        if step % 25 == 0:
            saver.save(sess, "make_checkpoint_folders/model.ckpt", global_step=step)
    print("Optimization Finished!")
    saver.save(sess, "final-model.ckpt", global_step=step)